In [1]:
# execute to import notebook styling for tables and width etc.
from IPython.core.display import HTML
import urllib.request
response = urllib.request.urlopen('https://raw.githubusercontent.com/DataScienceUWL/DS775v2/master/ds755.css')
HTML(response.read().decode("utf-8"));

In [2]:
# imports

from pyomo.environ import *
import string
import pandas as pd

<font size=18>Lesson 06 Homework</font>

# **HW6.1** - Textbook Problem 12.4-2

Reconsider the Wyndor Glass Co. problem presented in Sec. 3.1. Management now has decided that **only one** of the two new products should be produced, and the choice is to be made on the basis of maximizing profit. Introduce auxiliary binary variables to formulate an MIP model for this new version of the problem.

The model formulation as a MIP where $d$ and $w$ are the numbers of doors and windows produced, respectively, is:

Maximize $Z = 3 d + 5 w$

subject to:

$
\begin{array}{ccccc}
 d &   &    & \leq & 4 \\
   &   & 2w & \leq & 12 \\
3d & + & 2w & \leq & 18
\end{array}
$

$d \geq 0$, $w \geq 0$

**HW6.1a** - Introduce auxiliary binary variables to formulate a mixed BIP model for this problem. Include a picture or LaTeX of the mathematical formulation in the next cell.

<font color = "blue"> *** 4 points -  answer in cell below *** (don't delete this cell) </font>

<font color = "green">
Maximize $Z = 3 d + 5 w$

subject to:

$
\begin{array}{ccccc}
   &   &  d & \leq & 4 \\
   &   & 2w & \leq & 12 \\
3d & + & 2w & \leq & 18
\end{array}
$

$d \geq 0$, $w \geq 0$

$ d \leq M * y_1$
    
$ w \leq M * y_2$

$y_1 + y_2 = 1$
    
$y_1, y_2$ are binary

</font>

**HW6.1b** - Use Pyomo to solve this model. You can use a concrete or an abstract formulation.

<font color = "blue"> *** 6 points -  answer in cell below *** (don't delete this cell) </font>

In [3]:
m = ConcreteModel(name="problem_1")

# Variables
m.d = Var(bounds=(0,4))
m.w = Var(bounds=(0,6))

m.y1 = Var(domain=Boolean)
m.y2 = Var(domain=Boolean)

# Objective Function
m.profit = Objective( expr = 3*m.d + 5*m.w, sense = maximize)

# M
bigM = 10000

# Constraints:
m.cts = ConstraintList()
m.cts.add( 3 * m.d + 2 * m.w <= 18 )
m.cts.add( m.y1 + m.y2 == 1)
m.cts.add( m.d <= bigM * m.y1)
m.cts.add( m.w <= bigM * m.y2)

# Solve
solver = SolverFactory('glpk')
solver.solve(m)

# Output
print(f"Maximum Profit = ${1000*m.profit():,.2f}")
product = {1:"doors", 2:"windows"}
for i,amount in enumerate([m.d(),m.w()]):
    if amount > 0:
        print(f"Produce {amount:0.1f} {product[i+1]} per week")

Maximum Profit = $30,000.00
Produce 6.0 windows per week


# **HW6.2** - Textbook Problem 12.3-1

The Research and Development Division of the Progressive Company has been developing four possible new product lines. Management must now make a decision as to which of these four products actually will be produced and at what levels. Therefore, an operations research study has been requested to find the most profitable product mix.

A substantial cost is associated with beginning the production of any product, as given in the first row of the following table. Management’s objective is to find the product mix that maximizes the total profit (total net revenue minus start-up costs).

<img src="images/screen-prob12_3-1.png" alt="Solution" width="350" height="150">

Let the continuous decision variables $x_1, x_2, x_3,$ and $x_4$ be the production levels of products 1, 2, 3, and 4, respectively. Management has imposed the following policy constraints on these variables:

1. No more than two of the products can be produced.

2. Either product 3 or 4 can be produced only if either product 1 or 2 is produced.

3. Either 

$$5x_1 + 3x_2 + 6x_3 + 4x_4 \leq  6,000$$

$$\text{or}$$ 

$$4x_1 + 6x_2 + 3x_3 + 5x_4 \leq 6,000 $$.

**HW6.2a** - Introduce auxiliary binary variables to formulate a mixed BIP model for this problem.  Include a picture or LaTeX of the mathematical formulation in the next cell.

<font color = "blue"> *** 6 points -  answer in cell below *** (don't delete this cell) </font>

<font color='green'>
Maximize $Z = \sum{c_jx_j - k_jy_j}
\\$

Subject to:

$
\begin{array}{l}
x_j \leq My_j \\
y_1 + y_2 + y_3 + y_4 \leq 2 \\
y_5 + y_6 = 1 \\
 y_1 + y_2 + y_3 \leq 2 \\
5x_1 + 3x_2 + 6x_3 + 4x_4 \leq 6000 + M(1 - y_5) \\
4x_1 + 6x_2 + 3x_3 + 5x_4 \leq 6000 + M(1 - y_6) \\
y_3 - y_1 - y_2 \leq 0 \\
y_4 - y_1 - y_2 \leq 0 \\
x_i \geq 0, \text{ for } i=1, 2, 3, 4  \\
y_i \text{ binary, for } i=1, 2, 3, 4, 5, 6 \\
\\
where \\
c = \mbox{marginal cost} \\ 
x = \mbox{amount of product produced} \\ 
k = \mbox{startup cost of product} \\ 
y = \mbox{product was made (binary)} \\
M = \mbox{arbitrary large number} \\
\end{array}
$
   
</font>

**HW6.2b** - Use Pyomo to solve this model. Use an abstract formulation.

<font color = "blue"> *** 10 points -  answer in cell below *** (don't delete this cell) </font>

In [4]:
# Problem set-up
products = ['product 1', 'product 2', 'product 3', 'product 4']
marg_revenue = dict(zip(products, [70, 60, 90, 80]))
start_cost = dict(zip(products, [50000, 40000, 70000, 60000]))
choices = ['First','Second'] # y5, y6
bigM = 10000
coefs = [[5, 3, 6, 4], [4, 6, 3, 5]]
ys = [[-1,-1,1,0],[-1,-1,0,1]]
rhs = [6000, 6000]
max_products = 2
num_choices = 1

# --------------------------------------------------------
# Model
M = ConcreteModel(name="problem_2")

# --------------------------------------------------------
# Decision Variables
M.x = Var(products, domain=NonNegativeReals)
M.y = Var(products, domain=Boolean)
M.choice = Var(choices, domain=Boolean)

# --------------------------------------------------------
# Objective:  Maximize Profit
M.profit = Objective(expr=sum((marg_revenue[pr] * M.x[pr]) - start_cost[pr] * M.y[pr] for pr in products), sense=maximize)


# --------------------------------------------------------
# Constraints:
M.constraints = ConstraintList()

# fixed-charge
for pr in products:
    M.constraints.add(M.x[pr] <= bigM * M.y[pr])

# two products max
M.constraints.add(sum(M.y[pr] for pr in products) <= max_products)

# there can be only one   y5 + y6 == 1
M.constraints.add(sum(M.choice[ch] for ch in choices) == num_choices)

# the either/or constraint...
for i in range(len(choices)): 
    M.constraints.add(sum(coefs[i][j] * M.x[products[j]] for j in range(len(coefs[i]))) <= rhs[i] + bigM * (1 - M.choice[choices[i]]))

# product 3 or 4 can only be produced if either product 1 or 2 is produced
for i in range(len(ys)):
    M.constraints.add(sum(ys[i][j] * M.y[products[j]] for j in range(len(ys[i]))) <= 0)
    
# --------------------------------------------------------
# Solve
solver = SolverFactory('glpk')
solver.solve(M)

# --------------------------------------------------------
# Output
print(f"Maximum Profit = ${M.profit():,.2f}\n")
for pr in products:
    if bool(M.y[pr]()):
        print(f"Produce {pr} at a rate of {M.x[pr]():.0f} per week.")
    else:
        print(f"Do not produce {pr}." )
# curiosity
print("\nConstraint applied:")
for ch in choices:
    if M.choice[ch]:
        print(ch)

Maximum Profit = $80,000.00

Do not produce product 1.
Produce product 2 at a rate of 2000 per week.
Do not produce product 3.
Do not produce product 4.

Constraint applied:
First


# **HW6.3** - Textbook Problem 12.4-6

Speedy Delivery provides two-day delivery service of large parcels across the United States. Each morning at each collection center, the parcels that have arrived overnight are loaded onto several trucks for delivery throughout the area. Since the competitive battlefield in this business is speed of delivery, the parcels are divided among the trucks according to their geographical destinations to minimize the average time needed to make the deliveries.

On this particular morning, the dispatcher for the Blue River Valley Collection Center, Sharon Lofton, is hard at work. Her three drivers will be arriving in less than an hour to make the day’s deliveries. There are nine parcels to be delivered, all at locations many miles apart. As usual, Sharon has loaded these locations into her computer. She is using her company’s special software package, a decision support system called Dispatcher. The first thing Dispatcher does is use these locations to generate a considerable number of attractive possible routes for the individual delivery trucks. These routes are shown in the following table (where the numbers in each column indicate the order of the deliveries), along with the estimated time required to traverse the route.

<img src="images/screen-prob12_4-6.png" alt="Solution" width="350">


Dispatcher is an interactive system that shows these routes to Sharon for her approval or modification. (For example, the computer may not know that flooding has made a particular route infeasible.) After Sharon approves these routes as attractive possibilities with reasonable time estimates, Dispatcher next formulates and solves a BIP model for selecting three routes that minimize their total time while including each delivery location on exactly one route. This morning, Sharon does approve all the routes.

**HW6.3a** - Formulate this BIP model.  Include a picture or LaTeX of the mathematical formulation in the next cell.

<font color = "blue"> *** 6 points -  answer in cell below *** (don't delete this cell) </font>

<font color = "green">
Minimize $ Z = \sum_{j=1}^{10} t_iy_i
\\$

Subject to:

$
\begin{array}{l} 
\sum_{j=1}^{10}x_{ij}y_i = 1 \\
y_i \text{ binary, for } i=A:I \\
\\
where \\
t_i = \mbox{time (in hours) required to complete route} \\
i = \mbox{locations A:I} \\
j = \mbox{routes 1:10}
\\
\end{array}
$
</font>

**HW6.3b** - Use Pyomo solve this model.  Use an abstract formulation.

<font color = "blue"> *** 10 points -  answer in cell below *** (don't delete this cell) </font>

In [5]:
# Inputs
num_routes = 10
num_locations = 9
route_times = [6, 4, 7, 5, 4, 6, 5, 3, 7, 6]

# Setup
routes = ["Route " + str(i+1) for i in range(num_routes)]
alphabet = string.ascii_uppercase
locations = [alphabet[i] for i in range(num_locations)] # didn't end up needing this...
times = dict(zip(routes, route_times))
route_matrix = [[1,0,0,0,1,0,0,0,1,0],[0,1,0,1,0,1,0,0,1,1],[0,0,1,1,0,0,1,0,1,0],[1,0,0,0,0,1,0,1,0,0],[0,0,1,1,0,1,0,0,0,0],
                [0,1,0,0,1,0,0,0,0,0],[1,0,0,0,0,0,1,1,0,1],[0,0,1,0,1,0,0,0,0,1],[0,1,0,1,0,0,1,0,0,0]]

# Model
m = ConcreteModel(name="problem_3")

# Decision Variables
m.routes = Var(routes, domain=Boolean)

# Objective
m.min_times = Objective(expr=sum(times[r] * m.routes[r] for r in routes), sense=minimize)

# Constraints
m.constraints = ConstraintList()
for i in range(len(route_matrix)):
      m.constraints.add(expr=sum(route_matrix[i][r] * m.routes[routes[r]] for r in range(len(routes))) == 1)


# Solve
solver = SolverFactory('glpk')
solver.solve(m)

# Display
print("Use the following suggested routes:")
for r in routes:
    if m.routes[r]():
        print(r)
print(f"\nHours required: {m.min_times():,.2f}")

Use the following suggested routes:
Route 4
Route 5
Route 8

Hours required: 12.00


# **HW6.4** - Knapsack optimization

We'll revisit the Knapsack Optimization problem from Lesson 4 and 5. This time, we'll solve it using integer programming.

Given a set of items, each with a weight and a value, use binary variables and Pyomo to determine which items to include in a collection such that the total weight is less than or equal to a given limit and the total value is as large as possible. We will start with 20 items and you need to determine the collection of items that maximizes the value and keeps the total weight less than or equal to 50. 

Use the problem data as described below:

In [6]:
#Problem Data - generate random weights and values for a knapsack problem (do not change)
import numpy as np
num_items = 20
np.random.seed(seed=123)
values = np.random.randint(low=5, high=50, size=num_items)
weights = np.random.randint(low=1, high=10, size=num_items)
np.random.seed() # use system clock to reset the seed so future random numbers will appear random

Your Pyomo solution should go in the next cell.  Your code should still work if the number of items is changed to 40 or more.

<font color = "blue"> *** 8 points -  answer in cell below *** (don't delete this cell) </font>

In [7]:
# Model
m = ConcreteModel(name="problem_4")

# Decision variables
m.pack = Var(range(20),domain=Boolean) # <- 0's and 1's

# Objective
m.tot_value = Objective(expr=sum(m.pack[i] * values[i] for i in range(num_items)), sense=maximize)

# Constraints 
m.constraints = ConstraintList()
m.constraints.add(expr=sum(m.pack[i] * weights[i] for i in range(num_items)) <= 50)

# Solve
solver = SolverFactory('glpk')
solver.solve(m)

# Display
final_loadout = []
tot_weight = 0
for i in range(num_items):
    if m.pack[i]():
        final_loadout.append("Item: "+ str(i+1))
        tot_weight += weights[i]
print("Final loadout: ", final_loadout)
print(f"\nValue: {m.tot_value():.0f}")
print(f"\nWeight: ", tot_weight)

Final loadout:  ['Item: 1', 'Item: 3', 'Item: 4', 'Item: 7', 'Item: 8', 'Item: 9', 'Item: 10', 'Item: 12', 'Item: 13', 'Item: 15', 'Item: 17', 'Item: 18', 'Item: 19']

Value: 435

Weight:  50
